## Landscape

In [ ]:
import os
import subprocess

def has_landscape_config():
    # Check for a specific file or directory associated with the landscape module
    landscape_files = ['/etc/landscape/client.conf', '/etc/landscape/client.conf.d/']
    
    for file_or_dir in landscape_files:
        if os.path.exists(file_or_dir):
            return True
    
    return False

def search_landscape_config_entries():
    # Search for specific entries in configuration files
    landscape_keywords = ['landscape', 'some_other_keyword']
    config_files = ['/etc/landscape/client.conf', '/etc/landscape/client.conf.d/']

    for file_path in config_files:
        if os.path.exists(file_path):
            with open(file_path, 'r') as f:
                for line in f:
                    if any(keyword in line for keyword in landscape_keywords):
                        return True
    return False

def is_landscape_package_installed():
    # Use dpkg to check if the landscape package is installed
    result = subprocess.run("dpkg -l | grep landscape", shell=True, stdout=subprocess.PIPE, text=True)
    return "landscape" in result.stdout

def gather() -> dict:
    return {
        "Landscape": {
            "has_landscape_config": has_landscape_config(),
            "has_landscape_config_entries": search_landscape_config_entries(),
            "is_landscape_package_installed": is_landscape_package_installed(),
        }
    }
    
gather()

## APT Repositories

In [ ]:
import dataclasses
from pprint import pprint
from typing import Optional
import os
import subprocess

@dataclasses.dataclass
class AptRepository:
    archive_type: str
    options: Optional[dict[str, list[str]]]
    uri: str
    suite: str
    components: Optional[list[str]]

def get_apt_repositories() -> list[AptRepository]:
    # APT configuration files
    sources_list_path = '/etc/apt/sources.list'
    sources_list_d_path = '/etc/apt/sources.list.d/'

    repositories = []

    sources_list = []
    # Read main sources.list
    with open(sources_list_path, 'r') as sources_list_file:
        for line in sources_list_file:
            if line.startswith('deb'):
                sources_list.append(parse_repository_line(line.strip()))
    print(f"{len(sources_list)} repositories found in file: {sources_list_path}")
    repositories += sources_list

    sources_list_d = []
    # # Read files in sources.list.d directory
    for filename in os.listdir(sources_list_d_path):
        file_path = os.path.join(sources_list_d_path, filename)
        with open(file_path, 'r') as sources_list_d_file:
            for line in sources_list_d_file:
                if line.startswith('deb'):
                    sources_list_d.append(parse_repository_line(line.strip()))
    print(f"{len(sources_list_d)} repositories found in directory: {sources_list_d_path}")
    repositories += sources_list_d
    return repositories

def parse_repository_line(line: str) -> AptRepository:
    # Repo line format source: https://manpages.ubuntu.com/manpages/trusty/man5/sources.list.5.html
    repo_info = {}
    import re 
    repo_info["archive_type"] = line.split()[0]
    # if options exist
    if re.findall(r'\[.*?\]', line):
        repo_info["options"] = {}
        options_str = re.findall(r'\[(.*?)\]', line)[0]
        # parse each option and add to dict
        for option in options_str.strip().split():
            key, value = option.split("=")
            # if value is comma separated, split into list
            repo_info["options"][key] = value.split(",")
        # replace options string with empty string to make parsing rest of line easier
        line = line.replace(f"[{options_str}]", "")
    else:
        repo_info["options"] = None
    repo_info["uri"] = line.split()[1]
    repo_info["suite"] = " ".join(line.split()[2:min(4, len(line.split()))])
    if len(line.split()) > 4:
        repo_info["components"] = line.split()[4:]
    else:
        repo_info["components"] = None

    return AptRepository(**repo_info)

def get_apt_repositories_with_commands():
    # Use APT commands to get repositories
    result_main = subprocess.run("cat /etc/apt/sources.list", shell=True, stdout=subprocess.PIPE, text=True)
    result_d = subprocess.run("cat /etc/apt/sources.list.d/*", shell=True, stdout=subprocess.PIPE, text=True)

    repositories = result_main.stdout.strip().split('\n')
    repositories.extend(result_d.stdout.strip().split('\n'))

    return repositories

def gather() -> dict:
    return {
        "AptRepositories": {
            "repositories": get_apt_repositories(),
            "repositories_with_commands": get_apt_repositories_with_commands(),
        }
    }

pprint(gather()["AptRepositories"]["repositories"])
print(len(gather()["AptRepositories"]["repositories"]))

## APT Packages

In [ ]:
from pprint import pprint
import re
import subprocess
def get_installed_apt_packages_from_dpkg():
    # Run dpkg -l command to get a list of installed packages
    result = subprocess.run("dpkg -l", shell=True, stdout=subprocess.PIPE, text=True)

    # Parse the output to extract package names
    installed_packages = []
    lines = result.stdout.strip().split('\n')
    for line in lines[5:]:  # Skip the header lines
        package_info = line.split()
        if len(package_info) >= 2:
            package_name = package_info[1]
            installed_packages.append(package_name)

    return installed_packages

def get_installed_apt_packages_from_apt():
    result = subprocess.run("apt-mark showmanual", shell=True, stdout=subprocess.PIPE, text=True)
    return result.stdout.strip().split('\n')

def get_installed_apt_packages_from_history():
    result = subprocess.run("zgrep 'Commandline: apt' /var/log/apt/history.log /var/log/apt/history.log.*.gz", shell=True, stdout=subprocess.PIPE, text=True)
    # pprint(result.stdout.strip().split('\n'))
    # return result.stdout.strip().split('\n')
    packages = []
    for line in result.stdout.strip().split('\n'):
        if re.findall(r"Commandline: apt(-get)? install ", line):
            parsed_packages = [
                item for item in line.split("install", maxsplit=1)[1].split() 
                if not item.startswith("-")
            ]
            packages += parsed_packages
    packages.sort()
    return packages

def gather():

    # Return the results
    return {
        "AptPackages": {
            "packages_dpkg":  get_installed_apt_packages_from_dpkg(),
            "packages_apt": get_installed_apt_packages_from_apt(),
            "packages_apt_history": get_installed_apt_packages_from_history(),
        }
    }

result = gather()
print(len(result['AptPackages']['packages_dpkg']))
print(len(result['AptPackages']['packages_apt']))
print(len(result['AptPackages']['packages_apt_history']))
pprint(result['AptPackages']['packages_apt_history'])

## Users Config 

- get users config
- is user sudo?
- what groups is user in?
- what is user's shell?
- get content of user's bashrc or zshrc 
  - get zshrc completely duplicateable?  
- get user's ssh keys so they can later be used with ssh-import-id (ask more about import-id)

## Pro Client Config
- ask table flip server for what can actually be captured and then later configured using cloud-init

## Installed Snaps:

In [ ]:
def get_installed_snaps():
    # Run snap list command to get a list of installed snaps
    result = subprocess.run("snap list", shell=True, stdout=subprocess.PIPE, text=True)

    # Parse the output to extract snap names
    installed_snaps = []
    lines = result.stdout.strip().split('\n')
    for line in lines[1:]:  # Skip the header line
        snap_info = line.split()
        if len(snap_info) >= 2:
            snap_name = snap_info[0]
            installed_snaps.append(snap_name)

    return installed_snaps

# Example usage
installed_snaps = get_installed_snaps()
print("Installed Snaps:")
for snap in installed_snaps:
    print(snap)

## Home Directory Snapshotting

In [ ]:
import dataclasses
from glob import glob
from fnmatch import fnmatch

"""
Snapshot the home directory. Each HomeDirSnaphsotConfig object will 
contain the user's name, a list of files in home directory, and a list of 
directories in home directory. Each of the file/dir lists should be able to be 
wildcarded. Eeach should be defined via two lists: one to include and one to exclude. 
If an include entry has no wildcard, and matches one of the wildcarded entries in the
exclude list, the exclude list will take precedence (this is important for preventing 
secret files and whatnot from being snapshotted).
"""


@dataclasses.dataclass
class HomeDirSnapshotConfig:
    user: str
    include: list[str]
    exclude: list[str]  # this list will override the include list if there is a match
    max_depth: int = -1  # -1 means no limit


def snapshot_home_dir(config: HomeDirSnapshotConfig):
    root_dir = f"/home/{config.user}"
    # create tarball
    import tarfile

    added_files = []
    conflicts = []
    # create tarball with only the files/dirs specified in the config as include using wildcards
    # create this tarball in the current working directory
    exclude_matches = [glob(f"{root_dir}/{exclude_item}") for exclude_item in config.exclude]
    # flatten the list of lists
    exclude_matches = [item for sublist in exclude_matches for item in sublist]
    tarball_name = f"{config.user}_home_dir.tar.gz"
    with tarfile.open(tarball_name, "w:gz") as tar:
        for item in config.include:
            # Use glob to find files/directories matching the wildcard pattern
            matches = glob(f"{root_dir}/{item}")
            # eliminate matches that are in the exclude_matches list
            matches = [match for match in matches if match not in exclude_matches]
            for match in matches:
                # Check if the file matches any entry in the exclude list
                if any(fnmatch(match, exclude_item) for exclude_item in config.exclude):
                    conflicts.append(match)
                    continue  # Skip the file if it matches an exclude entry
                tar.add(match, recursive=True)
                added_files.append(match)
    print(f"Created tarball: {tarball_name}")
    print("Excluded the following files/directories that were matches for both the include and exclude lists:")
    for conflict in conflicts:
        print(f"- {conflict}")
    return tarball_name


my_config = HomeDirSnapshotConfig(
    user="a-dubs",
    include=[
        "Pictures/*",
        "Documents/*",
        "Desktop/*",
        "*.py",
        ".bashrc",
        ".bash_aliases",
        ".vimrc",
        ".zshrc",
    ],
    exclude=[
        "*.secret",
        "*.password",
        "*.key",
        "*.pem",
        "*.p12",
        "*.pfx",
        "Pictures/*.jpg",
    ],
)

snapshot_tar = snapshot_home_dir(my_config)